## Libraries

In [4]:
from collections import Counter,defaultdict
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from pprint import pprint
import json
from bs4 import BeautifulSoup
import sys
import os
import csv

#os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model, load_model
from keras.callbacks import Callback

from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

import re

In [35]:
typesTest = []

with open(r"C:\Users\PC-Axel\Documents\GitHub\thesis\Code\Data preparation\alleGemeentes2.csv", 'r') as csvfile:
    next(csvfile)
    spamreader = csv.reader(csvfile, delimiter='|')
    
    for row in spamreader:
        if "Raadsvragen" in row[3]:
            print(row)
        typesTest.append(row[3])
        #if "PGB" in row[2]:
        #    break


["b'gemeente-overbetuwe-overbetuwe-gld'", "b'Goedemorgen Arthur,  Enige weken geleden stelde ik een schriftelijke vraag over de aanvliegroutes van Lelystad over Overbetuws gebied. De routes zijn inmiddels gewijzigd. De vraag is dus overbodig geworden.    Met vriendelijke groeten,    Ard op de Weegh  '", "b'S-305 Schriftelijke vraag BBO over vliegverkeer van en naar Lelystad.'", "b'Raadsvragen'"]
["b'gemeente-overbetuwe-overbetuwe-gld'", "b'Beste collega van de griffie,    Ik heb binnen de organisatie het verzoek gekregen om beantwoording van de schriftelijke raadsvragen S-293 op mij te nemen.  Beantwoording dient te geschieden binnen 30 dagen. Vanwege vakanties van diverse betrokken collega\\xe2\\x80\\x99s, zwangerschapsverlof en waarneming per begin september achten we die termijn in dit specifieke geval niet haalbaar. Daar komt bij dat de eerste mogelijkheid om de concept beantwoording aan het college voor te leggen ook pas op 22 augustus is.     Onze inschatting is daarom dat de vra

["b'gemeente-overbetuwe-overbetuwe-gld'", "b'Vandaag schriftelijke vragen van het CDA ontvangen over MFC Valburg.  De gebruikelijke beantwoordingstermijn is 30 dagen.  Als gevolg van het kerstreces verwacht ik de beantwoording niet binnen de gestelde termijn voor elkaar te krijgen.  Wil je aan de vraagsteller doorgeven dat de beantwoording als gevolg van het kerstreces iets langer zal duren. Beantwoording zal voor eind januari 2018 plaatsvinden.    Met vriendelijke groet,    Roger Jakobs  Projectmanager  Team projectrealisatie  '", "b'S-317 Schriftelijke vragen CDA over voortgang MFC Valburg'", "b'Raadsvragen'"]


In [18]:
Counter(typesTest).most_common(50)

[("b'Agendapunt'", 7128),
 ("b'Commissiebrieven S&R'", 2056),
 ("b'Verslag'", 1937),
 ("b'Toezeggingen S&R'", 1783),
 ("b'Toezeggingen M&S'", 1734),
 ("b'Commissiebrieven M&S'", 1682),
 ("b'Toezeggingen Raad'", 1113),
 ("b'Agenda'", 1063),
 ("b'Schriftelijke vragen'", 740),
 ("b'Uitnodigingen'", 146),
 ("b'Report'", 139),
 ("b'Moties'", 130),
 ("b'agendapunt_actielijst'", 101),
 ("b'Gemeentebladen'", 97),
 ("b'Raadsvragen'", 94),
 ("b'Videotulen Commissie Stad en Ruimte'", 81),
 ("b'Besluitenlijst'", 78),
 ("b'Videotulen Commissie Mens en Samenleving'", 73),
 ("b'Resolution'", 69),
 ("b'Videotulen Raadsvergadering '", 68),
 ("b'Amendementen'", 67),
 ("b'Videotulen Gemeenteraad'", 63),
 ("b'Informatieve Stukken College'", 55),
 ("b'Toezeggingen'", 46),
 ("b'Videotulen Vragenuur'", 33),
 ("b'Videotulen Commissie Beleid Algemeen'", 31),
 ("b'Videotulen Commissie Samenleving'", 27),
 ("b'Commissiebrieven C&F'", 26),
 ("b'Videotulen Commissie Ruimte'", 26),
 ("b'Videotulen Gemeenteraadsverg